In [1]:
import pandas as pd
import numpy as np

## 10.1 Risk Parity, Normal Assumption

In [2]:
df = pd.read_csv('test5_2.csv')
df

,x1,x2,x3,x4,x5
0,0.084979,0.116781,0.042304,0.008984,0.003876
1,0.116781,0.160485,0.058136,0.012345,0.005326
2,0.042304,0.058136,0.037440,0.005963,0.002573
3,0.008984,0.012345,0.005963,0.001688,0.000546
4,0.003876,0.005326,0.002573,0.000546,0.000314


In [ ]:
def risk_parity_weights(cov_matrix, tol=1e-6, max_iter=100):
    n = cov_matrix.shape[0]
    y = np.ones(n)
    for i in range(max_iter):
        g = cov_matrix @ y - 1.0 / y
        H = cov_matrix + np.diag(1.0 / (y**2))
        dy = np.linalg.solve(H, -g)
        y = y + dy
        if np.linalg.norm(dy) < tol:
            break
    w = y / np.sum(y)
    return w

In [10]:
df = pd.read_csv('test5_2.csv')
cov = df.values
weights = risk_parity_weights(cov)
pd.DataFrame(weights)

,0
0,0.035464
1,0.025806
2,0.056469
3,0.265920
4,0.616341


In [11]:
ans = pd.read_csv('testout10_1.csv')
ans

,W
0,0.035464
1,0.025806
2,0.056469
3,0.265920
4,0.616341


## 10.2 Risk Parity, Normal Assumption, 1/2 risk weight on X5

In [12]:
df = pd.read_csv('test5_2.csv')
df

,x1,x2,x3,x4,x5
0,0.084979,0.116781,0.042304,0.008984,0.003876
1,0.116781,0.160485,0.058136,0.012345,0.005326
2,0.042304,0.058136,0.037440,0.005963,0.002573
3,0.008984,0.012345,0.005963,0.001688,0.000546
4,0.003876,0.005326,0.002573,0.000546,0.000314


In [14]:
def risk_budgeting_weights(cov_matrix, budgets, tol=1e-6, max_iter=100):
    n = cov_matrix.shape[0]
    y = 1.0 / np.sqrt(np.diag(cov_matrix))
    assert np.all(budgets > 0)
    for i in range(max_iter):
        g = cov_matrix @ y - budgets / y
        H = cov_matrix + np.diag(budgets / (y**2))
        dy = np.linalg.solve(H, -g)
        step = 1.0
        while np.any(y + step * dy <= 0):
            step *= 0.5
            if step < 1e-5:
                break
        y = y + step * dy
        if np.linalg.norm(step * dy) < tol:
            break
    w = y / np.sum(y)
    return w


In [15]:
cov = df.values
budgets = np.array([1.0, 1.0, 1.0, 1.0, 0.5])
weights = risk_budgeting_weights(cov, budgets)
pd.DataFrame(weights)

,0
0,0.050242
1,0.036560
2,0.080399
3,0.378608
4,0.454191


In [ ]:
ans = pd.read_csv('testout10_2.csv')
ans

,W
0,0.050242
1,0.036560
2,0.080399
3,0.378608
4,0.454191


## 10.3 Max Sharpe Ratio, normal assumption, w>0

In [16]:
df = pd.read_csv('test5_2.csv')
df

,x1,x2,x3,x4,x5
0,0.084979,0.116781,0.042304,0.008984,0.003876
1,0.116781,0.160485,0.058136,0.012345,0.005326
2,0.042304,0.058136,0.037440,0.005963,0.002573
3,0.008984,0.012345,0.005963,0.001688,0.000546
4,0.003876,0.005326,0.002573,0.000546,0.000314


In [17]:
rf = 0.04

In [18]:
means = pd.read_csv("test10_3_means.csv")
means

,Mean
0,0.09
1,0.08
2,0.07
3,0.06
4,0.05


In [21]:

def project_to_simplex(v):
    n = len(v)
    u = np.sort(v)[::-1]
    cssv = np.cumsum(u)
    rho = np.nonzero(u * np.arange(1, n+1) > (cssv - 1))[0][-1]
    theta = (cssv[rho] - 1) / (rho + 1.0)
    w = np.maximum(v - theta, 0)
    return w

def min_variance_simplex(cov_matrix, tol=1e-9, max_iter=50000):
    n = cov_matrix.shape[0]
    x = np.ones(n) / n
    L = np.linalg.norm(cov_matrix, ord=2)
    lr = 1.0 / L
    y = x.copy()
    t = 1.0
    
    for i in range(max_iter):
        grad = cov_matrix @ y
        x_new = project_to_simplex(y - lr * grad)
        if np.linalg.norm(x_new - x) < tol:
            return x_new
        t_new = (1.0 + np.sqrt(1.0 + 4.0 * t * t)) / 2.0
        y = x_new + ((t - 1.0) / t_new) * (x_new - x)
        x = x_new
        t = t_new
    return x

def max_sharpe_ratio_numpy(cov_matrix, expected_returns, rf=0.0):
    excess_returns = expected_returns - rf
    if np.any(excess_returns <= 0):
        print("Warning: Some excess returns are non-positive.")
        
    mus = excess_returns.reshape(-1, 1)
    mu_matrix = mus @ mus.T
    sigma_tilde = cov_matrix / mu_matrix
    
    z = min_variance_simplex(sigma_tilde)
    y = z / excess_returns
    w = y / np.sum(y)
    return w


In [22]:
cov = pd.read_csv('test5_2.csv').values
means = pd.read_csv('test10_3_means.csv')['Mean'].values
weights = max_sharpe_ratio_numpy(cov, means, rf=0.04)
pd.DataFrame(weights)

,0
0,0.00000
1,0.00000
2,0.00000
3,0.12138
4,0.87862


In [20]:
ans = pd.read_csv('testout10_3.csv')
ans

,W
0,-9.997589e-09
1,-9.998604e-09
2,-9.996571e-09
3,1.213108e-01
4,8.786893e-01


## 10.4 Max Sharpe Ratio, normal assumption, 0.1 <= w <= 0.5

In [43]:
df = pd.read_csv('test5_2.csv')
df

,x1,x2,x3,x4,x5
0,0.084979,0.116781,0.042304,0.008984,0.003876
1,0.116781,0.160485,0.058136,0.012345,0.005326
2,0.042304,0.058136,0.037440,0.005963,0.002573
3,0.008984,0.012345,0.005963,0.001688,0.000546
4,0.003876,0.005326,0.002573,0.000546,0.000314


In [55]:
means = pd.read_csv('test10_3_means.csv')['Mean'].values

In [57]:

def project_to_box_constraints(v, lower, upper, target_sum=1.0):
    def sum_weights(lam):
        return np.sum(np.clip(v - lam, lower, upper))
    lam_min = np.min(v) - np.max(upper) - 10.0
    lam_max = np.max(v) - np.min(lower) + 10.0
    for _ in range(100):
        lam_mid = (lam_min + lam_max) / 2.0
        s = sum_weights(lam_mid)
        if np.abs(s - target_sum) < 1e-9:
            break
        if s > target_sum:
            lam_min = lam_mid
        else:
            lam_max = lam_mid
    return np.clip(v - lam_mid, lower, upper)

def max_sharpe_ratio_pgd(cov_matrix, expected_returns, rf=0.04, min_w=0.1, max_w=0.5, max_iter=10000, tol=1e-9):
    n = cov_matrix.shape[0]
    w = np.ones(n) / n
    lr = 0.1
    decay = 0.9995
    y = w.copy()
    t = 1.0
    for i in range(max_iter):
        mu_py = y @ expected_returns
        sigma_py = np.sqrt(y @ cov_matrix @ y)
        sharpe_y = (mu_py - rf) / sigma_py
        grad_y = (1.0 / sigma_py) * (sharpe_y * (cov_matrix @ y) / sigma_py - expected_returns)
        w_new = project_to_box_constraints(y - lr * grad_y, min_w, max_w)
        if np.linalg.norm(w_new - w) < tol:
            return w_new
        t_new = (1.0 + np.sqrt(1.0 + 4.0 * t * t)) / 2.0
        y = w_new + ((t - 1.0) / t_new) * (w_new - w)
        w = w_new
        t = t_new
        lr *= decay
        
    return w

In [58]:
cov = df.values
weights = max_sharpe_ratio_pgd(cov, means, rf=0.04, min_w=0.1, max_w=0.5)
pd.DataFrame(weights)

,0
0,0.1
1,0.1
2,0.1
3,0.5
4,0.2


In [25]:
ans = pd.read_csv('testout10_4.csv')
ans

,W
0,0.1
1,0.1
2,0.1
3,0.5
4,0.2


## 11.1 Expost Attribution

In [69]:
returns = pd.read_csv('test11_1_returns.csv')
weights = pd.read_csv('test11_1_weights.csv')

In [72]:
returns

,x1,x2,x3
0,0.027468,-0.026885,-0.009229
1,-0.040896,0.068180,0.072992
2,0.019978,0.021086,0.014608
3,0.025690,-0.037519,-0.031387
4,-0.029897,0.008592,-0.013029
5,-0.054385,0.014818,0.037323
6,0.000312,-0.020412,-0.002518
7,-0.026667,-0.005633,0.005474
8,0.003628,-0.028907,-0.019324
9,0.046772,0.015728,-0.028908


In [79]:
weights

,W
0,0.3
1,0.2
2,0.5


In [84]:
cum_returns_ext

array([[1.        , 1.        , 1.        ],
       [1.02746765, 0.97311485, 0.99077062],
       [0.98544869, 1.03946218, 1.06308924],
       [1.00513619, 1.06137993, 1.07861923],
       [1.03095813, 1.02155761, 1.04476409],
       [1.00013576, 1.03033475, 1.03115203],
       [0.94574348, 1.04560256, 1.06963746],
       [0.94603822, 1.02425965, 1.06694442],
       [0.92080977, 1.01849003, 1.0727848 ],
       [0.9241504 , 0.98904821, 1.05205385],
       [0.96737467, 1.00460359, 1.02164084],
       [0.96142964, 1.03212756, 1.02481257],
       [0.98102369, 1.06291404, 1.05099153],
       [0.97544508, 1.10201525, 1.06065574],
       [0.94116117, 1.08706954, 1.080101  ],
       [0.92029354, 1.06497477, 1.06447821],
       [0.92462296, 1.0581264 , 1.11104234],
       [0.89942635, 1.07362098, 1.12541153],
       [0.86772181, 1.06449078, 1.13347121],
       [0.83423639, 1.07005903, 1.20215842],
       [0.84972063, 1.07105484, 1.25619182],
       [0.85603148, 1.06912138, 1.24300429],
       [0.

In [92]:
asset_values

array([[0.3       , 0.2       , 0.5       ],
       [0.3082403 , 0.19462297, 0.49538531],
       [0.29563461, 0.20789244, 0.53154462],
       [0.30154086, 0.21227599, 0.53930962],
       [0.30928744, 0.20431152, 0.52238205],
       [0.30004073, 0.20606695, 0.51557602],
       [0.28372305, 0.20912051, 0.53481873],
       [0.28381147, 0.20485193, 0.53347221],
       [0.27624293, 0.20369801, 0.5363924 ],
       [0.27724512, 0.19780964, 0.52602693],
       [0.2902124 , 0.20092072, 0.51082042],
       [0.28842889, 0.20642551, 0.51240628],
       [0.29430711, 0.21258281, 0.52549576],
       [0.29263352, 0.22040305, 0.53032787],
       [0.28234835, 0.21741391, 0.5400505 ],
       [0.27608806, 0.21299495, 0.53223911],
       [0.27738689, 0.21162528, 0.55552117],
       [0.26982791, 0.2147242 , 0.56270576],
       [0.26031654, 0.21289816, 0.5667356 ],
       [0.25027092, 0.21401181, 0.60107921],
       [0.25491619, 0.21421097, 0.62809591],
       [0.25680944, 0.21382428, 0.62150214],
       [0.

In [98]:
w_init = weights['W'].values
n_t, n_assets = returns.shape
cum_returns = (1 + returns).cumprod()
cum_returns_ext = np.vstack([np.ones(n_assets), cum_returns])
asset_values = cum_returns_ext * w_init
port_values = asset_values.sum(axis=1)

port_returns_bh = port_values[1:] / port_values[:-1] - 1
R_p = port_values[-1] - 1
w_actual = asset_values[:-1] / port_values[:-1][:, None]

K = R_p / np.log(1 + R_p) if R_p != 0 else 1.0
k_t = np.zeros(n_t)
mask = (port_returns_bh == 0)
k_t[mask] = 1.0
k_t[~mask] = np.log(1 + port_returns_bh[~mask]) / port_returns_bh[~mask]
coeff = K * k_t
attr_return = []
for i in range(n_assets):
    r_it = returns.iloc[:, i].values
    term = coeff * w_actual[:, i] * r_it
    attr_return.append(np.sum(term))
sigma_p = port_returns_bh.std(ddof=1)
contribs = w_actual * returns.values
attr_vol = []
for i in range(n_assets):
    c_i = contribs[:, i]
    cov_i = np.cov(c_i, port_returns_bh, ddof=1)[0, 1]
    attr_vol.append(cov_i / sigma_p)

# 5. Display Results
print("Return:", asset_values[-1,:]/asset_values[0,:]-1, R_p)
print("Return Attribution:", attr_return, sum(attr_return))
print("Vol Attribution:", attr_vol, sum(attr_vol))

Return: [-0.22144565 -0.01600753  0.30146696] 0.081098280073725
Return Attribution: [-0.06551252231430578, -0.002219805321367216, 0.14883060770939763] 0.08109828007372463
Vol Attribution: [-0.0006202226315012855, 0.002827104435466038, 0.012579817544960906] 0.01478669934892566


In [67]:
ans = pd.read_csv('testout11_1.csv')
ans

,Value,x1,x2,x3,Portfolio
0,TotalReturn,-0.221446,-0.016008,0.301467,0.081098
1,Return Attribution,-0.065513,-0.002220,0.148831,0.081098
2,Vol Attribution,-0.000620,0.002827,0.012580,0.014787


## 11.2 Expost Attribution to Factors

In [100]:
stock_returns = pd.read_csv('test11_2_stock_returns.csv')
factor_returns = pd.read_csv('test11_2_factor_returns.csv')
weights_df = pd.read_csv('test11_2_weights.csv')
betas_df = pd.read_csv('test11_2_beta.csv')

In [ ]:
w_init = weights_df['W'].values
betas = betas_df.iloc[:, 1:].values

n_t, n_assets = stock_returns.shape
n_factors = factor_returns.shape[1]
cum_returns = (1 + stock_returns).cumprod()
cum_returns_ext = np.vstack([np.ones(n_assets), cum_returns])
asset_values = cum_returns_ext * w_init
port_values = asset_values.sum(axis=1)
port_returns_bh = port_values[1:] / port_values[:-1] - 1
R_p = port_values[-1] - 1

w_actual = asset_values[:-1] / port_values[:-1][:, None]

beta_p = w_actual @ betas

factor_contribs = beta_p * factor_returns.values

alpha_contribs = port_returns_bh - factor_contribs.sum(axis=1)

total_return_factors = (1 + factor_returns).prod() - 1
total_return_alpha = (1 + alpha_contribs).prod() - 1
total_return_port = R_p

K = R_p / np.log(1 + R_p) if R_p != 0 else 1.0
k_t = np.zeros(n_t)
mask = (port_returns_bh == 0)
k_t[mask] = 1.0
k_t[~mask] = np.log(1 + port_returns_bh[~mask]) / port_returns_bh[~mask]
coeff = K * k_t

attr_factors = []
for j in range(n_factors):
    term = coeff * factor_contribs[:, j]
    attr_factors.append(np.sum(term))

term_alpha = coeff * alpha_contribs
attr_alpha = np.sum(term_alpha)

sigma_p = port_returns_bh.std(ddof=1)

vol_attr_factors = []
for j in range(n_factors):
    c_j = factor_contribs[:, j]
    cov_j = np.cov(c_j, port_returns_bh, ddof=1)[0, 1]
    vol_attr_factors.append(cov_j / sigma_p)

cov_alpha = np.cov(alpha_contribs, port_returns_bh, ddof=1)[0, 1]
vol_attr_alpha = cov_alpha / sigma_p



In [121]:
print("Total Return:", total_return_factors.values, total_return_alpha, total_return_port)
print("Return Attribution:", attr_factors, attr_alpha)
print('Vol Attribution:', vol_attr_factors, vol_attr_alpha, sum(vol_attr_factors))

Total Return: [ 0.07993903  0.02671084 -0.01631719] 0.04644303819357587 0.10532667972875176
Return Attribution: [0.06517280614688023, 0.00017358190287437755, -0.007578621692838648] 0.04755891337183552
Vol Attribution: [0.005012796084655259, -9.192566178651305e-06, 0.0017847245024119574] 0.005445186530229761 0.006788328020888565


In [99]:
ans = pd.read_csv('testout11_2.csv')
ans

,Value,F1,F2,F3,Alpha,Portfolio
0,TotalReturn,0.079939,0.026711,-0.016317,0.046443,0.105327
1,Return Attribution,0.065173,0.000174,-0.007579,0.047559,0.105327
2,Vol Attribution,0.005013,-0.000009,0.001785,0.005445,0.012234
